In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit


In [ ]:
file_path = r'C:\Users\Alavis1\Documents\Works\Budgetsets/subset_brand2_loc1_reg3_chan1.csv'
data = pd.read_csv(file_path)
data['FISCAL_DATE'] = pd.to_datetime(data['FISCAL_DATE'])
data = data[['FISCAL_DATE', '$ Advertising Cost', '$ Profit']]
print("Extracted Data:")
print(data.head())

In [ ]:
def linear_regression(x, a, b):
    return a * x + b

def michaelis_menten(x, a, b):
    return a * x / (b + x)

def logarithmic_regression(x, a, b):
    return a * np.log(x) + b

def gompertz_curve(x, a, b, c):
    return a * np.exp(-b * np.exp(-c * x))

def exponential_rise(x, a, b):
    return a * np.exp(b * x)

def generalized_logistics(x, a, b, c):
    return a / (1 + np.exp(-b * (x - c)))

seasons = {i: data[data['FISCAL_DATE'].dt.month == i] for i in range(1, 13)}

def fit_and_calculate_mse(season_data, model_func):
    x = season_data['$ Advertising Cost'].values
    y = season_data['$ Profit'].values
    try:
        popt, _ = curve_fit(model_func, x, y, maxfev=10000)
        y_pred = model_func(x, *popt)
        mse = mean_squared_error(y, y_pred)
        return mse, popt
    except RuntimeError as e:
        print(f"Model fitting failed for {model_func.name} with error: {e}")
        return float('inf'), None

mse_results = {'Linear': 0, 'Michaelis-Menten': 0, 'Logarithmic': 0, 'Gompertz': 0, 'Exponential Rise': 0, 'Generalized Logistics': 0}
model_functions = {
    'Linear': linear_regression,
    'Michaelis-Menten': michaelis_menten,
    'Logarithmic': logarithmic_regression,
    'Gompertz': gompertz_curve,
    'Exponential Rise': exponential_rise,
    'Generalized Logistics': generalized_logistics
}

for season, season_data in seasons.items():
    if not season_data.empty:
        mse_results['Linear'] += fit_and_calculate_mse(season_data, linear_regression)[0]
        mse_results['Michaelis-Menten'] += fit_and_calculate_mse(season_data, michaelis_menten)[0]
        mse_results['Logarithmic'] += fit_and_calculate_mse(season_data, logarithmic_regression)[0]
        mse_results['Gompertz'] += fit_and_calculate_mse(season_data, gompertz_curve)[0]
        mse_results['Exponential Rise'] += fit_and_calculate_mse(season_data, exponential_rise)[0]
        mse_results['Generalized Logistics'] += fit_and_calculate_mse(season_data, generalized_logistics)[0]

print("MSE Results:")
print(pd.DataFrame([mse_results]))

In [ ]:
best_model = min(mse_results, key=mse_results.get)
print("Best Model:", best_model)

In [ ]:
def calculate_seasonality_function(model_func, popt1, popt2, prev_month, next_month):
    if model_func == linear_regression:
        a1, _ = popt1
        a2, _ = popt2
        return f"S = {a2} / {a1}"
    elif model_func == michaelis_menten:
        a1, b1 = popt1
        a2, b2 = popt2
        return f"S(x) = ({a2} * {b2}) / ({a1} * {b1}) * (({b1} + x{prev_month})^2 / ({b2} + x{next_month})^2)"
    elif model_func == logarithmic_regression:
        a1, _ = popt1
        a2, _ = popt2
        return f"S(x) = ({a2} * x{prev_month}) / ({a1} * x{next_month})"
    elif model_func == gompertz_curve:
        a1, b1, c1 = popt1
        a2, b2, c2 = popt2
        return f"S(x) = ({a2} * {b2} * {c2} * exp(-{b2} * exp(-{c2} * x{next_month})) * exp(-{c2} * x{next_month})) / ({a1} * {b1} * {c1} * exp(-{b1} * exp(-{c1} * x{prev_month})) * exp(-{c1} * x{prev_month}))"
    elif model_func == exponential_rise:
        a1, b1 = popt1
        a2, b2 = popt2
        return f"S(x) = ({a2} * {b2} * exp({b2} * x{next_month})) / ({a1} * {b1} * exp({b1} * x{prev_month}))"
    elif model_func == generalized_logistics:
        a1, b1, c1 = popt1
        a2, b2, c2 = popt2
        return f"S(x) = ({a2} * {b2} * exp(-{b2} * (x{next_month} - {c2})) / (1 + exp(-{b2} * (x{next_month} - {c2})))^2) / ({a1} * {b1} * exp(-{b1} * (x{prev_month} - {c1})) / (1 + exp(-{b1} * (x{prev_month} - {c1})))^2)"

seasonality_factors = {}
for i in range(1, 13):
    next_i = 1 if i == 12 else i + 1
    if not seasons[i].empty and not seasons[next_i].empty:
        mse1, popt1 = fit_and_calculate_mse(seasons[i], model_functions[best_model])
        mse2, popt2 = fit_and_calculate_mse(seasons[next_i], model_functions[best_model])
        if popt1 is not None and popt2 is not None:
            seasonality_factors[f'{i}-{next_i}'] = calculate_seasonality_function(model_functions[best_model], popt1, popt2, i, next_i)
        else:
            seasonality_factors[f'{i}-{next_i}'] = "Model fitting failed"

print("Seasonality Factors as Functions of X:")
for key, value in seasonality_factors.items():
    print(f"{key}: {value}")